In [165]:
import osmnx as ox
import geopandas as gpd
import os
import pandas as pd
from utils.osmgraph import GraphFunctions
from utils.spaitalShape import Point, OdPair, Box
import numpy as np
import plotly.graph_objects as go
import plotly

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [18]:
ox.__version__

'0.16.1'

In [19]:
roadRestrictionFilefolder = 'data/restrictionByMnDoT/'

In [45]:
#read gdfs
gdf = None
for root, dirs, files in os.walk(roadRestrictionFilefolder):
    for file in files:
        if file[-5] == '7' or file[-5] == '5':
            filepath = os.path.join(root, file)
            if gdf is None:
                gdf = ox.geometries.geometries_from_xml(filepath)
            else:
                gdf = gpd.GeoDataFrame(pd.concat([gdf, ox.geometries.geometries_from_xml(filepath)], ignore_index=True), crs=gdf.crs)   
roadRestriction = gdf.dropna(subset=['length_m']).reset_index(drop=True)         

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
roadRestriction.head()

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,unique_id,osmid,element_type,nodes,DELETE,sort,from_tm,to_tm,sll_zone,name,from_rp,to_rp,description,posting_ton,length_m,geometry,sll_znoe,id
0,way/-97459856,-97459856,way,"[-99379200, -99380400, -99378360, -99378840, -...",5,0300000000000117-I,0.00000000000,1.79800000000,CENTRAL,MN 117 (7 ton),0+00.000,1+00.797,MN 117 from ND-MN state line to Jct. MN 27,7,2894.31901029000,"LINESTRING (-96.63923 45.76899, -96.63373 45.7...",NaN,NaN
1,way/-97461512,-97461512,way,"[-111202800, -111203280, -111203400, -11120244...",13,0300000000000007-I,0.00000000000,24.08600000000,CENTRAL,MN 7 (7 ton),0+00.000,24+00.173,MN 7 from Jct. MN 28 to Golf Club Road,7,38762.59877150000,"LINESTRING (-96.70002 45.55872, -96.70002 45.5...",NaN,NaN
2,way/-97460072,-97460072,way,"[-111470112, -111468672, -111470832, -11146771...",23,0300000000000022-I,157.15400000000,162.61500000000,CENTRAL,MN 22 (7 ton),156+00.901,162+00.372,MN 22 from MN 55 (Eden Valley) to Stearns CSAH-9,7,8788.95858247000,"LINESTRING (-94.54656 45.32252, -94.54657 45.3...",NaN,NaN
3,way/-97345512,-97345512,way,"[-112343096, -112333616, -112333736, -11233385...",1,0300000000000210-I,218.62500000000,227.85500000000,NORTH-CENTRAL,MN 210 (7 ton),218+00.558,227+000.790,MN 210 from MN 45 (Carlton) to west MN 23 (Dul...,7,14853.70816620000,"LINESTRING (-92.42485 46.66397, -92.42266 46.6...",NaN,NaN
4,way/-97345728,-97345728,way,"[-114610464, -114611064, -114610584, -11461070...",16,0300000000000123-I,0.00000000000,8.04100000000,NORTH-CENTRAL,MN 123 (7 ton),0+00.000,8+00.054,MN 123 from south Jct. MN 23 (Sandstone) to no...,7,12941.01403050000,"LINESTRING (-92.87985 46.12517, -92.87555 46.1...",NaN,NaN


In [48]:
distance = 70
distance = distance*1609.34 # mile->km
bbox = ox.utils_geo.bbox_from_point((44.9827, -93.22025), dist=distance, project_utm = False, return_crs = False)
boundingBox = Box(bbox[-1], bbox[-2], bbox[-3], bbox[-4])
print(boundingBox)

-94.65258355209289,-91.7879164479071,43.96958146789451,45.995818532105496


c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
osmGraphInBbox = GraphFunctions.extractGraphOf(boundingBox)

Graphs/GraphDataInBbox-94.65258355209289,-91.7879164479071,43.96958146789451,45.995818532105496
reloading graph..


In [51]:
nodes, edges = osmGraphInBbox.graphToGdfs()

In [55]:
len(nodes)

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


223490

In [56]:
len(edges)

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


570087

In [60]:
pathlist = list(roadRestriction['geometry'])

In [71]:
pathListInBbox = []
for path in pathlist:
    if path.covered_by(boundingBox.polygon()):
        pathListInBbox.append(path)

In [72]:
len(pathListInBbox)

10

In [97]:
len(list(pathListInBbox[0].coords))

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


53

In [76]:
bridgeRestrictionDF = pd.read_excel('data/Bridge_Restrictions.xlsx')

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [77]:
bridgeRestrictionDF.head()

,BRKEY,ON_UNDER,FACILITY,ROADWAY_NAME,FEATURE_INTERSECTED,DISTRICT,COUNTY,CITY - TOWNSHIP,LOCATION DESC,OWNER,...,PERMIT_CODE_48K_DESC,PERMIT_CODE_60K_CODE,PERMIT_CODE_60K_DESC,PERMIT_CODE_72K_CODE,PERMIT_CODE_72K_DESC,VEHICLE_POSTED_CAPACITY,SEMI_POSTED_CAPACITY,DOUBLE_POSTED_CAPACITY,LATITUDE,LONGITUDE
0,01002,1,MN 200,MN 200,HILL LAKE CHANNEL,01,AITKIN,HILL LAKE (TOWNSHIP OF),0.8 MI E OF JCT TH 169,STATE HIGHWAY AGENCY,...,NO RESTRICTION,1,NO RESTRICTION,1,NO RESTRICTION,NaN,NaN,NaN,46.987514,-93.588928
1,01004,1,US 169,US 169,MISSISSIPPI RIVER,03,AITKIN,MORRISON (TOWNSHIP OF),2.9 MI N OF N JCT TH 210,STATE HIGHWAY AGENCY,...,NO RESTRICTION,1,NO RESTRICTION,1,NO RESTRICTION,NaN,NaN,NaN,46.651128,-93.612747
2,01005,1,MN 210,MN 210,RIPPLE RIVER,03,AITKIN,AITKIN,0.2 MI E OF W JCT TH 169,STATE HIGHWAY AGENCY,...,NO RESTRICTION,1,NO RESTRICTION,1,NO RESTRICTION,NaN,NaN,NaN,46.533364,-93.702469
3,01006,1,MN 210,MN 210,RICE RIVER,03,AITKIN,MORRISON (TOWNSHIP OF),7.4 MI NE OF JCT TH 169,STATE HIGHWAY AGENCY,...,NO RESTRICTION,1,NO RESTRICTION,1,NO RESTRICTION,NaN,NaN,NaN,46.598764,-93.612883
4,01007,1,MN 210,MN 210,BNSF RR,01,AITKIN,MCGREGOR,0.5 MI E OF W JCT TH 65,STATE HIGHWAY AGENCY,...,NO RESTRICTION,1,NO RESTRICTION,1,NO RESTRICTION,NaN,NaN,NaN,46.609233,-93.307086


In [84]:
bridgeRestrictionDF['VERTICAL_CLR_RD1_FEET']

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
24381     NaN
24382    18.0
24383    18.0
24384     NaN
24385    17.4
Name: VERTICAL_CLR_RD1_FEET, Length: 24386, dtype: float64

In [89]:
values = {"VERTICAL_CLR_RD1_FEET": 100, "VERTICAL_CLR_RD2_FEET": 100, "HORIZONTAL_CLR_RD1_FEET": 100, "HORIZONTAL_CLR_RD2_FEET": 100}
bridgeRestrictionDF = bridgeRestrictionDF.fillna(value=values)

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
bridgeRestrictionDF.iloc[0]

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


BRKEY                                        01002
ON_UNDER                                         1
FACILITY                                    MN 200
ROADWAY_NAME                                MN 200
FEATURE_INTERSECTED              HILL LAKE CHANNEL
DISTRICT                                        01
COUNTY                                      AITKIN
CITY - TOWNSHIP            HILL LAKE (TOWNSHIP OF)
LOCATION DESC               0.8 MI E OF JCT TH 169
OWNER                         STATE HIGHWAY AGENCY
VERTICAL_CLR_RD1_FEET                        100.0
VERTICAL_CLR_RD2_FEET                        100.0
HORIZONTAL_CLR_RD1_FEET                       45.9
HORIZONTAL_CLR_RD2_FEET                      100.0
PERMIT_CODE_48K_CODE                             1
PERMIT_CODE_48K_DESC                NO RESTRICTION
PERMIT_CODE_60K_CODE                             1
PERMIT_CODE_60K_DESC                NO RESTRICTION
PERMIT_CODE_72K_CODE                             1
PERMIT_CODE_72K_DESC           

In [135]:
bridgeRestriction = bridgeRestrictionDF[(bridgeRestrictionDF['PERMIT_CODE_72K_CODE']=='X') 
                    | ((bridgeRestrictionDF['VERTICAL_CLR_RD1_FEET'] < 13.5) & (bridgeRestrictionDF['VERTICAL_CLR_RD1_FEET'] >0.1 ))
                    | ((bridgeRestrictionDF['VERTICAL_CLR_RD2_FEET'] < 13.5) & (bridgeRestrictionDF['VERTICAL_CLR_RD2_FEET'] >0.1 ))
                    | ((bridgeRestrictionDF['HORIZONTAL_CLR_RD1_FEET'] < 8.5) & (bridgeRestrictionDF['HORIZONTAL_CLR_RD1_FEET'] >0.1 ))
                    | ((bridgeRestrictionDF['HORIZONTAL_CLR_RD2_FEET'] < 8.5) & (bridgeRestrictionDF['HORIZONTAL_CLR_RD2_FEET'] >0.1 ))]

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [136]:
len(bridgeRestriction)

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


957

In [137]:
def isInBbox(arraylike):
    point = Point(arraylike['LONGITUDE'], arraylike['LATITUDE'])
    if point.isContained(boundingBox):
        return True
    return False

In [138]:
bridgeRestriction['inBbox'] = bridgeRestriction.apply(isInBbox, axis=1)

<ipython-input-138-71c82ce07507>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bridgeRestriction['inBbox'] = bridgeRestriction.apply(isInBbox, axis=1)


In [139]:
bridgeRestriction.head()

,BRKEY,ON_UNDER,FACILITY,ROADWAY_NAME,FEATURE_INTERSECTED,DISTRICT,COUNTY,CITY - TOWNSHIP,LOCATION DESC,OWNER,...,PERMIT_CODE_60K_CODE,PERMIT_CODE_60K_DESC,PERMIT_CODE_72K_CODE,PERMIT_CODE_72K_DESC,VEHICLE_POSTED_CAPACITY,SEMI_POSTED_CAPACITY,DOUBLE_POSTED_CAPACITY,LATITUDE,LONGITUDE,inBbox
12,01502,1,CSAH 22,CSAH 22 (400TH ST),WAKEFIELD BROOK,03,AITKIN,AITKIN (TOWNSHIP OF),1.2 MI W OF JCT CSAH 24,COUNTY HIGHWAY AGENCY,...,1,NO RESTRICTION,X,DENIED,NaN,NaN,NaN,46.593289,-93.777664,False
165,02502,1,CSAH 28,CSAH 28,SEELYE BROOK,METRO,ANOKA,ST FRANCIS,0.1 MI W OF JCT CR 71,COUNTY HIGHWAY AGENCY,...,X,DENIED,X,DENIED,32.0,NaN,NaN,45.402922,-93.417939,True
288,02J26,1,MSAS 143,LAKES PKWY NE,LAKE CHANNEL,METRO,ANOKA,BLAINE,0.9 MI E OF JCT CSAH 52,CITY OR MUNICIPAL HIGHWA,...,X,DENIED,X,DENIED,NaN,NaN,NaN,45.192647,-93.195672,True
290,02J28,1,MUN 1053,EDISON ST,LAKE CHANNEL,METRO,ANOKA,BLAINE,0.1 MI S OF LAKE BLVD,CITY OR MUNICIPAL HIGHWA,...,X,DENIED,X,DENIED,NaN,NaN,NaN,45.180892,-93.195700,True
324,03504,1,T 676,150TH AVE,BUFFALO RIVER,04,BECKER,CUBA (TOWNSHIP OF),1.0 MI N OF JCT CSAH 12,TOWN OR TOWNSHIP HIGHWAY,...,1,NO RESTRICTION,X,DENIED,NaN,NaN,NaN,46.948506,-96.090844,False


In [140]:
print(boundingBox)

-94.65258355209289,-91.7879164479071,43.96958146789451,45.995818532105496


c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [141]:
bridgeRestriction = bridgeRestriction[bridgeRestriction['inBbox']]

In [142]:
len(bridgeRestriction)

240

In [143]:
bridgeRestriction.describe(include=[np.number])

,VERTICAL_CLR_RD1_FEET,VERTICAL_CLR_RD2_FEET,HORIZONTAL_CLR_RD1_FEET,HORIZONTAL_CLR_RD2_FEET,VEHICLE_POSTED_CAPACITY,SEMI_POSTED_CAPACITY,DOUBLE_POSTED_CAPACITY,LATITUDE,LONGITUDE
count,240.000000,240.0,240.000000,240.000000,185.000000,137.000000,137.000000,240.000000,240.000000
mean,98.487083,100.0,30.183750,98.750000,25.281081,35.591241,35.605839,44.760023,-93.341723
std,11.649410,0.0,12.325343,9.836105,9.188658,6.648258,6.669022,0.545344,0.651383
min,7.600000,100.0,8.000000,0.000000,3.000000,14.000000,14.000000,43.974286,-94.639983
25%,100.000000,100.0,24.000000,100.000000,20.000000,32.000000,32.000000,44.288313,-93.831548
50%,100.000000,100.0,28.000000,100.000000,26.000000,40.000000,40.000000,44.717263,-93.285110
75%,100.000000,100.0,32.000000,100.000000,32.000000,40.000000,40.000000,45.035478,-92.899417
max,100.000000,100.0,100.000000,100.000000,40.000000,40.000000,40.000000,45.976231,-91.805811


In [147]:
pathListInBbox[0].coords[0]

(-94.54655583, 45.32252174)

In [152]:
restristedPointList = list(zip(list(bridgeRestriction['LATITUDE']), list(bridgeRestriction['LONGITUDE'])))

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [153]:
restristedPointList[0]

(45.402922, -93.417939)

In [155]:
GraphFunctions.plotPointList({'restrict nodes':  restristedPointList}, ['red'], 'test.html')

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [160]:
trajList = []
for i in pathListInBbox:
    trajList.append(list(i.coords))
print(len(trajList)), print(len(trajList[0]))

10
53


(None, None)

In [161]:
print(trajList[0])

[(-94.54655583, 45.32252174), (-94.54656984, 45.32390502), (-94.54655619, 45.32427189), (-94.54657932, 45.32512165), (-94.54659109, 45.32598824), (-94.54659314, 45.32718583), (-94.5465619, 45.32811682), (-94.54655397, 45.3283355), (-94.54651701, 45.32935598), (-94.54641734, 45.33291583), (-94.5462951, 45.33468286), (-94.54624518, 45.33604311), (-94.54613328, 45.33994672), (-94.54650922, 45.36152701), (-94.54684036, 45.36723391), (-94.54677746, 45.36798637), (-94.54646379, 45.36833718), (-94.54643228, 45.36837194), (-94.54587602, 45.36864668), (-94.54515965, 45.36873289), (-94.53913402, 45.36884577), (-94.53904943, 45.36888183), (-94.53850917, 45.36911495), (-94.53812391, 45.36947598), (-94.53786733, 45.37015453), (-94.53780221, 45.37106455), (-94.53755982, 45.37186893), (-94.53708973, 45.37220832), (-94.53654328, 45.3723822), (-94.53559427, 45.3724122), (-94.53480911, 45.37243702), (-94.53421645, 45.3725845), (-94.53375401, 45.37288326), (-94.53346826, 45.37340019), (-94.53350677, 45.3

c:\Users\mingzhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [168]:
class GraphFunctionTest():
    @staticmethod
    def __initGoFigure(lat, long, type='markers', label='default', size=5, color='red'):
        '''
        type = 'markers' or 'lines'
        lat/long: list, if 'type == lines'; float, else
        '''
        fig = go.Figure(go.Scattermapbox(
            name=label,
            mode=type,
            lon=long,
            lat=lat,
            marker={'size': size, 'color': color}))
        return fig

    @staticmethod
    def __addTrace(fig, lat, long, type='markers', label='default', size=5, color='red'):
        fig.add_trace(go.Scattermapbox(
            name=label,
            mode=type,
            lon=long,
            lat=lat,
            marker={'size': size, 'color': color}))
        return fig

    @staticmethod
    def __plotFigAndSave(fig, lat_center, long_center, filename):
        fig.update_layout(mapbox_style="stamen-terrain",
                          mapbox_center_lat=30, mapbox_center_lon=-80)
        fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0},
                          mapbox={
                              'center': {'lat': lat_center,
                                         'lon': long_center},
                              'zoom': 9.5})
        plotly.offline.plot(fig, filename=filename, auto_open=False)
        return

    @staticmethod
    def plotTrajList(trajList, color, label, filename, order = 0):
        '''
        filename: string end with ".html"
        order: 0=>(lat, lon), 1=>(lon,lat)
        '''
                
        i = 0
        for traj in trajList:
            if order == 0:
                lat, long = zip(*traj)
            else:
                long, lat = zip(*traj)
            # adding the lines joining the nodes
            if i == 0:
                fig = GraphFunctionTest.__initGoFigure(lat, long, type='lines', label=label, size=5, color=color)

            else:
                fig = GraphFunctionTest.__addTrace(fig, lat, long, type='lines', label=label, size=5, color=color)
            i += 1
        # getting center for plots:
        lat_center = np.mean(lat)
        long_center = np.mean(long)
        GraphFunctionTest.__plotFigAndSave(fig, lat_center, long_center, filename)

In [169]:
GraphFunctionTest.plotTrajList(trajList, 'red', 'restrictedPath', 'restrictedPath.html',1)

red
red
red
red
red
red
red
red
red
